In [15]:
import pandas as pd
import numpy as np

In [16]:
movies  = pd.read_csv('/content/tmdb_5000_credits.csv')


In [17]:
credits =pd.read_csv('/content/tmdb_5000_movies.csv')


In [18]:
movies = movies.merge(credits,on='title')


In [19]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [20]:
movies.isnull().sum()

,0
movie_id,0
title,0
overview,3
genres,0
keywords,0
cast,0
crew,0


In [21]:
movies.dropna(inplace=True)

In [22]:
import ast
def convert(obj):
  l = []
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [23]:
import ast
def convertCast(obj):
  l = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter != 3:
      l.append(i['name'])
      counter+=1
    else:
      break
  return l


In [24]:
import ast
def fetch_Director(obj):
  l = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      l.append(i['name'])
      break
  return l

In [25]:
movies['genres'] = movies['genres'].apply(convert)

In [26]:
movies['keywords'] = movies['keywords'].apply(convert)

In [27]:
movies['cast'] = movies['cast'].apply(convertCast)

In [28]:
movies['crew'] = movies['crew'].apply(fetch_Director)

In [35]:
movies['genres'] =movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] =movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] =movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] =movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [38]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [39]:
movies['tags'] = movies['overview']+ movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [42]:
new_df = movies[['movie_id', 'title', 'tags']]

In [43]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

/tmp/ipython-input-3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [45]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

/tmp/ipython-input-3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [46]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [ ]:
!pip install nltk

**NLTK libaray is use to just add the main words of each not all like if there is loving , love , loved so it will one consider it as one word love**

In [65]:
import nltk

In [66]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [67]:
def stem(text):
  y = []
  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words = 'english')

In [85]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [86]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [87]:
from sklearn.metrics.pairwise import cosine_similarity

In [114]:
similarity = cosine_similarity(vectors)

In [89]:
similarity

array([[1.        , 0.08740748, 0.05827165, ..., 0.02418254, 0.02564946,
        0.        ],
       [0.08740748, 1.        , 0.06451613, ..., 0.02677398, 0.        ,
        0.        ],
       [0.05827165, 0.06451613, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.02418254, 0.02677398, 0.02677398, ..., 1.        , 0.07071068,
        0.04836508],
       [0.02564946, 0.        , 0.        , ..., 0.07071068, 1.        ,
        0.05129892],
       [0.        , 0.        , 0.        , ..., 0.04836508, 0.05129892,
        1.        ]])

In [115]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [118]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem


In [119]:
import pickle

In [122]:
pickle.dump(new_df,open('movie_list.pkl','wb'))


In [123]:
pickle.dump(similarity,open('similarity.pkl','wb'))